In [29]:
import pandas as pd
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

bbc = pd.read_csv('bbc.csv')
bbc.head()
print(bbc.shape)
print(bbc.info())
print(bbc.head())   

(2225, 2)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    2225 non-null   object
 1   topic   2225 non-null   object
dtypes: object(2)
memory usage: 34.9+ KB
None
                                                text     topic
0  Dallaglio his own man to the end\r\n\r\nContro...     sport
1  Best person' for top legal job\r\n\r\nThe "bes...  politics
2  Viewers to be able to shape TV\r\n\r\nImagine ...      tech
3  Fox attacks Blair's Tory 'lies'\r\n\r\nTony Bl...  politics
4  Microsoft debuts security tools\r\n\r\nMicroso...      tech


In [30]:
# Check for missing data and print the total number of missing values for each column
print("Missing data:", bbc.isnull().sum())

# Check for duplicate rows in the dataset and print the total number of duplicate rows
print("Duplicate data:", bbc.duplicated().sum())

# Display the unique values of the 'topic' column to examine the different categories
print("Unique values in the 'topic' column:", bbc['topic'].unique())

Missing data: text     0
topic    0
dtype: int64
Duplicate data: 98
Unique values in the 'topic' column: ['sport' 'politics' 'tech' 'business' 'entertainment']


- There are 98 duplicate rows in the dataset.
- There is no missing data.
- There are 5 unique values in the 'topic' column, and none of them are incorrect or inconsistent.
- Therefore, we only need to remove the duplicate rows.

In [31]:
# Display summary statistics of the cleaned dataset
bbc.describe()
print(bbc.head())


                                                text     topic
0  Dallaglio his own man to the end\r\n\r\nContro...     sport
1  Best person' for top legal job\r\n\r\nThe "bes...  politics
2  Viewers to be able to shape TV\r\n\r\nImagine ...      tech
3  Fox attacks Blair's Tory 'lies'\r\n\r\nTony Bl...  politics
4  Microsoft debuts security tools\r\n\r\nMicroso...      tech


In [32]:
bbc["text"] = bbc["text"].str.lower()
print(bbc.head())

                                                text     topic
0  dallaglio his own man to the end\r\n\r\ncontro...     sport
1  best person' for top legal job\r\n\r\nthe "bes...  politics
2  viewers to be able to shape tv\r\n\r\nimagine ...      tech
3  fox attacks blair's tory 'lies'\r\n\r\ntony bl...  politics
4  microsoft debuts security tools\r\n\r\nmicroso...      tech


In [33]:
bbc.loc[:, "text"] = bbc["text"].str.replace("'s", "", regex=True)
bbc

,text,topic
0,dallaglio his own man to the end\r\n\r\ncontro...,sport
1,"best person' for top legal job\r\n\r\nthe ""bes...",politics
2,viewers to be able to shape tv\r\n\r\nimagine ...,tech
3,fox attacks blair tory 'lies'\r\n\r\ntony blai...,politics
4,microsoft debuts security tools\r\n\r\nmicroso...,tech
...,...,...
2220,michael film signals 'retirement'\r\n\r\nsinge...,entertainment
2221,ray charles studio becomes museum\r\n\r\na mus...,entertainment
2222,chancellor rallies labour voters\r\n\r\ngordon...,politics
2223,oscar nominees gear up for lunch\r\n\r\nleonar...,entertainment


In [34]:
# Loại bỏ dấu câu và ký tự đặc biệt
# Loại bỏ các ký tự đặc biệt trong cột 'text'
bbc["text"] = bbc["text"].str.replace(r"[;!?,.:()\"']", "", regex=True)
bbc

,text,topic
0,dallaglio his own man to the end\r\n\r\ncontro...,sport
1,best person for top legal job\r\n\r\nthe best ...,politics
2,viewers to be able to shape tv\r\n\r\nimagine ...,tech
3,fox attacks blair tory lies\r\n\r\ntony blair ...,politics
4,microsoft debuts security tools\r\n\r\nmicroso...,tech
...,...,...
2220,michael film signals retirement\r\n\r\nsinger ...,entertainment
2221,ray charles studio becomes museum\r\n\r\na mus...,entertainment
2222,chancellor rallies labour voters\r\n\r\ngordon...,politics
2223,oscar nominees gear up for lunch\r\n\r\nleonar...,entertainment


In [35]:
# Create set of stop words from sklearn
stop_words = set(ENGLISH_STOP_WORDS)

# This function is use for removing stop words in text 
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return ' '.join(filtered_words)

bbc.loc[:, 'text'] = bbc['text'].apply(remove_stopwords)

bbc.head()

,text,topic
0,dallaglio man end controversy lawrence dallagl...,sport
1,best person legal job best person job appointe...,politics
2,viewers able shape tv imagine editing titanic ...,tech
3,fox attacks blair tory lies tony blair lied to...,politics
4,microsoft debuts security tools microsoft rele...,tech


In [36]:
import re

# Hàm để loại bỏ các từ viết tắt như "&", "/",":"
def remove_abbreviations_and_slash(text):
    abbreviations = r'\b(&|/)\b'
    cleaned_text = re.sub(abbreviations, '', text)
    # Loại bỏ khoảng trắng thừa sau khi xóa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    return cleaned_text

# Áp dụng hàm cho cột 'text' trong DataFrame
bbc.loc[:,'text'] = bbc['text'].apply(remove_abbreviations_and_slash)
bbc

,text,topic
0,dallaglio man end controversy lawrence dallagl...,sport
1,best person legal job best person job appointe...,politics
2,viewers able shape tv imagine editing titanic ...,tech
3,fox attacks blair tory lies tony blair lied to...,politics
4,microsoft debuts security tools microsoft rele...,tech
...,...,...
2220,michael film signals retirement singer george ...,entertainment
2221,ray charles studio museum museum dedicated car...,entertainment
2222,chancellor rallies labour voters gordon brown ...,politics
2223,oscar nominees gear lunch leonardo dicaprio ja...,entertainment


In [37]:

from nltk.stem import WordNetLemmatizer

# Tạo lemmatizer
lemmatizer = WordNetLemmatizer()
bbc["text"] = bbc["text"].apply(lambda x: ' '.join([lemmatizer.lemmatize(word, 'v') for word in x.split()]))
bbc

,text,topic
0,dallaglio man end controversy lawrence dallagl...,sport
1,best person legal job best person job appoint ...,politics
2,viewers able shape tv imagine edit titanic wat...,tech
3,fox attack blair tory lie tony blair lie take ...,politics
4,microsoft debut security tool microsoft releas...,tech
...,...,...
2220,michael film signal retirement singer george m...,entertainment
2221,ray charles studio museum museum dedicate care...,entertainment
2222,chancellor rally labour voters gordon brown is...,politics
2223,oscar nominees gear lunch leonardo dicaprio ja...,entertainment


In [38]:

if 'text' in bbc.columns:
    # Hàm để loại bỏ số và đơn vị tiền tệ
    def remove_currency_and_numbers_with_logging(text):
        removed_items = re.findall(r'\b\d+\b|\$\w*|%\w*|₫\w*|â£\w*|£\w*|â\w*', text)
        cleaned_text = re.sub(r'\b\d+\b|\$\w*|%\w*|₫\w*|â£\w*|£\w*|â\w*', '', text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        return cleaned_text, removed_items
    
    # Áp dụng hàm cho cột 'text' và lưu lại các mục bị loại bỏ
    removed_data = []
    def process_text(row):
        cleaned_text, removed_items = remove_currency_and_numbers_with_logging(row)
        removed_data.extend(removed_items)  # Lưu các mục bị loại bỏ
        return cleaned_text

    # Sử dụng .loc để thay đổi giá trị trong DataFrame
    bbc.loc[:, 'text'] = bbc['text'].apply(process_text)

    # Lưu danh sách các mục bị loại bỏ vào một biến
    removed_data = list(set(removed_data))  # Loại bỏ các mục trùng lặp

    # Xuất danh sách các mục bị loại bỏ
    print(f"Số lượng từ bị xóa: {len(removed_data)}")
    print("Các mục bị loại bỏ:", removed_data)

Số lượng từ bị xóa: 2287
Các mục bị loại bỏ: ['$512bn', '$6717bn', '$7500', '$295bn', '783', '$3920', 'â£931m', '787', '2010', '1149', 'â£198m', '34882', 'â£154m', '417', 'â£175m', 'â£8000', '357', '920000', '257', '273', '$325m', '74', '$2115', 'â£391bn', '$26bn', '178000', '$618m', '$5115', '32000', '$332', '324', '1974', 'â£279bn', 'â£341m', '20032004', 'â£13700', '1980', '170', '120000', 'â£80000', '$53m', 'â£600000', 'â£74m', '$9358m', '51075', '150000', '£158m', 'â£600m', '001', '102', '$140m', '$1791m', 'â£33', 'â£55bn', '$1102m', '$21bn', 'â£5bn', 'â£234m', '1875', '$1364', '$152', '$40', 'â£13bn', 'â£680m', 'â£95m', 'â£7m', 'â£500m', '$45m', 'â£928bn', '$17m', '0', 'â£115000', '4987', 'â£63', 'â£428m', '$1834bn', 'â£220', '3000', '$142m', '$284bn', '1972', '200001', 'â£340000', '$175m', 'â£39bn', '$565m', '70000', '$63', 'â£307m', '2', '0130', '800000', 'â£134bn', '41911', 'â£13500', '$249m', 'â£34bn', '78', '102975', '85130', '$280', '500000', '497', '$80m', '$13280', '801', 

In [39]:

from collections import Counter

# Kiểm tra sự tồn tại của cột 'text'
if 'text' in bbc.columns:
    # Đếm tần suất của các từ
    all_words = ' '.join(bbc['text']).split()
    word_counts = Counter(all_words)

    # Lọc ra các từ xuất hiện từ 5 lần trở lên
    common_words = {word for word, count in word_counts.items() if count >= 5}
    
    # In số lượng từ phổ biến
    print(f"Số lượng từ phổ biến: {len(common_words)}")

    # Danh sách từ bị xóa (từ hiếm gặp)
    removed_words = {word for word in word_counts.keys() if word not in common_words}
    
    # In danh sách các từ hiếm gặp
    print(f"Số lượng từ bị xóa: {len(removed_words)}")

    bbc.loc[:, 'text'] = bbc['text'].apply(lambda x: ' '.join([word for word in x.split() if word in common_words]))

Số lượng từ phổ biến: 8790
Số lượng từ bị xóa: 17375


In [40]:

# Kiểm tra sự tồn tại của cột 'text'
if 'text' in bbc.columns:
    # Hàm để loại bỏ dấu gạch ngang '-' không liên kết với từ
    def clean_text(text):
        # Loại bỏ dấu gạch ngang không có từ đi kèm
        cleaned_text = re.sub(r'(?<=\s)-(?=\s)', '', text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
        return cleaned_text
    
    # Áp dụng hàm cho cột 'text'
    bbc.loc[:, 'text'] = bbc['text'].apply(clean_text)
bbc


,text,topic
0,dallaglio man end controversy lawrence dallagl...,sport
1,best person legal job best person job appoint ...,politics
2,viewers able shape tv imagine edit titanic wat...,tech
3,fox attack blair tory lie tony blair lie take ...,politics
4,microsoft debut security tool microsoft releas...,tech
...,...,...
2220,michael film signal retirement singer george m...,entertainment
2221,ray charles studio museum museum dedicate care...,entertainment
2222,chancellor rally labour voters gordon brown is...,politics
2223,oscar nominees gear leonardo dicaprio jamie fo...,entertainment


In [41]:
# Find duplicate rows
# Tìm các dòng trùng lặp dựa trên cột 'text'
duplicate_rows = bbc[bbc.duplicated(subset=['text'], keep=False)]

# Hiển thị các nhóm dữ liệu trùng
grouped_duplicates = duplicate_rows.sort_values(by=['text']).groupby('text')
for group, rows in grouped_duplicates:
    print(f"Duplicate group: {group}")
    print(rows)

print(duplicate_rows)

# Start cleaning the data by removing duplicate rows
bbc = bbc.drop_duplicates()

print(bbc)

Duplicate group: 2d metal slug offer retro fun like drill sergeant past metal slug today gamers slick visuals fancy 2d consider retro release years ago frantic shooter end year yes include halo simply choose 2d level video game blast encounter toughest game likely play enemies live pile pressure players battle soldier zombies giant alien mention huge boss guard level gameplay pepper moments genius fan robotic note title refer instead vast array vehicles offer game stuff bizarre hardware tank jet offer iraq justice joy thank ultra control tough crack addictive mere metal slug cheap slice fry man say course ignore lack do visual modern time blockbuster title offer fresh paint favour real innovation metal slug fresh air era xbox gate eye
                                                   text topic
1325  2d metal slug offer retro fun like drill serge...  tech
1314  2d metal slug offer retro fun like drill serge...  tech
Duplicate group: apple attack source row civil liberties group electr

In [42]:
# Save the cleaned dataset to a new CSV file
bbc.to_csv('bbc_cleaned.csv', index=False)